In [ ]:
import csv
from pathlib import Path
import json

import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter

import stock

class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND * 0.2)),
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)

In [ ]:
ticker = yf.Ticker("MSFT", session=session)
history = ticker.history(interval="1wk", period="2y")

close_values = history.Close.to_numpy()
ma10 = np.convolve(close_values, np.ones(10) / 10, mode="valid")
ma30 = np.convolve(close_values, np.ones(30) / 30, mode="valid")
ma40 = np.convolve(close_values, np.ones(40) / 40, mode="valid")

plt.plot(close_values)
plt.plot([i for i in range(10 - 1, len(close_values))], ma10)
plt.plot([i for i in range(30 - 1, len(close_values))], ma30)
plt.plot([i for i in range(40 - 1, len(close_values))], ma40)


In [ ]:
def strength(target: np.array):
    pct_change = np.diff(target) / target[:-1]
    return np.cumprod((1 + pct_change))[-1] - 1


def relative_strength(target: np.array,  reference: np.array) -> float:
    target_strength = strength(target)
    ref_strength = strength(reference)
    rs = (target_strength + 1) / (ref_strength + 1) * 100
    return rs


def relative_strength_52wk(target: np.array, reference: np.array, num_division: int = 52, division_factor:float = 1.02):
    weights = np.array([division_factor ** i for i in range(num_division)], dtype=float)
    weights /= np.linalg.norm(weights, ord=1)

    rs = 0.0
    for i in range(num_division):
        start = max(i * len(target) // num_division - 1, 0)
        end = min((i + 1) * len(target) // num_division + 1, len(target))
        # print(start, end)
        rs += weights[i] * relative_strength(target[start:end], reference[start:end])
    return rs

In [ ]:
sp500_history = yf.Ticker("^GSPC").history(interval="1wk", period="2y")
sp500_closes = sp500_history.Close.to_numpy()
dow_history = yf.Ticker("^DJI").history(interval="1wk", period="2y")
dow_closes = dow_history.Close.to_numpy()
nasdaq_history = yf.Ticker("^IXIC").history(interval="1wk", period="2y")
nasdaq_closes = nasdaq_history.Close.to_numpy()

In [ ]:
ticker.quarterly_financials

In [ ]:
def trand_template_for_stage2(history: pd.DataFrame) -> bool:
    """第2ステージを特定するためのトレンドテンプレート
    """
    close_values = history.Close.to_numpy()
    if len(close_values) < 60:
        return False
    ma10 = np.convolve(close_values, np.ones(10) / 10, mode="valid")
    ma30 = np.convolve(close_values, np.ones(30) / 30, mode="valid")
    ma40 = np.convolve(close_values, np.ones(40) / 40, mode="valid")
    current_value = close_values[-1]

    low52 = np.min(history.Low.to_numpy()[-52:])
    high52 = np.max(history.High.to_numpy()[-52:])

    flag = True
    # 現在の株価が30週移動平均線と40週移動平均線を上回っている
    flag &= current_value > ma40[-1] and current_value > ma30[-1]
    # 150日(30週)移動平均線が200日(40週)移動平均線を上回っている
    flag &= ma30[-1] > ma40[-1]
    # 200日(40週)移動平均線は少なくとも1ヶ月上昇トレンドにある
    flag &= np.all([ma40[-i - 1] - ma40[-i -2] > 0 for i in range(4)])
    # 50日(10週)移動平均線は150日(30週)移動平均線と200日(40週)移動平均線を上回っている
    flag &= ma10[-1] > ma30[-1] and ma10[-1] > ma40[-1]
    # 現在の株価は50日移動平均線を上回っている
    flag &= current_value > ma10[-1]
    # 現在の株価は52週安値より少なくとも30%高い
    flag &= current_value > 1.3 * low52
    # 現在の株価は52週高値から少なくとも25%以内にある
    flag &= current_value > 0.75 * high52
    
    return flag

def fundamentals_template_for_stage2(ticker: yf.Ticker) -> bool:
    financials = ticker.financials
    flag = True
    # 直近2四半期の利益がプラス
    try:
        flag &= financials.loc["Diluted EPS"][0] > 0 and financials.loc["Diluted EPS"][1] > 0
    except:
        stock.logger.exception("Failed to get Diluted EPS")
        flag = False
    # 直近2四半期のepsが前年同期比で20%以上上昇
    
    return flag

def calculate_relative_strengths(history: pd.DataFrame, num_division: int = 12, division_factor=1.1) -> tuple[float, float, float]:
    close_values = history.Close.to_numpy()
    if len(close_values) < 52:
        return 0.0

    # relative strength 
    rs_sp500 = relative_strength_52wk(close_values[-52:], sp500_closes[-52:], num_division=num_division, division_factor=division_factor)
    rs_dow = relative_strength_52wk(close_values[-52:], dow_closes[-52:], num_division=num_division, division_factor=division_factor)
    rs_nasdaq = relative_strength_52wk(close_values[-52:], nasdaq_closes[-52:], num_division=num_division, division_factor=division_factor)

    return rs_sp500, rs_dow, rs_nasdaq

def save_data(ticker: yf.Ticker, output_dir: Path):
    """Save quarterly financial fundamentals.
    """
    output_dir.mkdir(exist_ok=True, parents=True)
    output_path = output_dir / f"{ticker.ticker}.json"

    financials = {}
    if output_path.exists():
        with open(output_path, "r") as f:
            try:
                financials = json.load(f)
            except:
                stock.logger.exception("Failed to load json data")

    for key, value in ticker.quarterly_financials.to_dict().items():
        financials[key.strftime("%Y-%m-%d")] = value
    with open(output_path, "w") as f:
        json.dump(financials, f, indent=4)
    stock.logger.info(f"Save data : {output_path.as_posix()}")

In [ ]:
# load stock code list
with open(stock.DATA_DIR / "us_stock_codes.csv", "r") as f:
    csv_reader = csv.reader(f)
    all_codes = [row[0] for row in list(csv_reader)[1:]]

flags = np.array([False for i in range(len(all_codes))], dtype=bool)
relative_strengths = np.zeros((len(all_codes), 3), dtype=float)
for idx, code in enumerate(all_codes):
    stock.logger.info(f"Start checking : {code}")
    ticker = yf.Ticker(code, session=session)
    history = ticker.history(interval="1wk", period="2y")

    flags[idx] = trand_template_for_stage2(history) and fundamentals_template_for_stage2(ticker)
    relative_strengths[idx, :] = calculate_relative_strengths(history)

    save_data(ticker, stock.DATA_DIR / "codes")
    # stock.logger.info(f"!!!!Good ticker!!!! : {code}")

In [ ]:
rss = np.mean(relative_strengths, axis=1)
pt80 = np.percentile(rss, 80)
good = np.logical_and(flags, rss > pt80)
good.sum()

In [ ]:
for idx in range(len(flags)):
    if good[idx]:
        code = all_codes[idx]
        percentile = (rss < rss[idx]).sum() / len(rss)
        ticker = yf.Ticker(code)
        print(f"{code}")

In [ ]:
for key in ticker.quarterly_financials.to_dict().keys():
    key.strftime("%Y-%m-%d")

In [ ]:
code = "AAL"
ticker = yf.Ticker(code, session=session)
history = ticker.history(interval="1wk", period="2y")

data = history.Close.to_numpy()
data52 = data[-52:]
sp52 = sp500_closes[-52:]
dow52 = dow_closes[-52:]
nas52 = nasdaq_closes[-52:]

plt.plot(data52 / data52[0], label="target")
plt.plot(sp52 / sp52[0], label="sp500")
plt.plot(dow52 / dow52[0], label="dow")
plt.plot(nas52 / nas52[0], label="nasdaq")
#plt.plot(sp500_closes / sp500_closes[0], label="sp500")
#plt.plot(dow_closes / dow_closes[0], label="dow")
#plt.plot(nasdaq_closes / nasdaq_closes[0], label="nasdaq")
plt.legend()

calculate_relative_strengths(history) #, num_division=1, division_factor=0)